In [1]:
!pip install -r "C:\Users\memo_\Desktop\Projeler\AutoRAG-End-to-End-RAG-Framework-with-Turkish-Embedding-Evaluation\requirements.txt"



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'

In [1]:
import glob
import docx
import nltk
import transformers
import torch
import spacy
import numpy as np
import logging
import unstructured
import unstructured.partition.auto
import zipfile
import time

c:\Users\memo_\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

# Transformers'ın TensorFlow modüllerini yüklemesini tamamen engelliyoruz:
os.environ["USE_TF"] = "0"
os.environ["TRANSFORMERS_NO_TF"] = "1"

# Şimdi güvenle diğer kütüphaneleri import edebilirsiniz:
print("TensorFlow'u devre dışı bırakma ayarları yapıldı.")
print(f"USE_TF: {os.getenv('USE_TF')}")
print(f"TRANSFORMERS_NO_TF: {os.getenv('TRANSFORMERS_NO_TF')}")

try:
    from qdrant_client import QdrantClient
    from nltk.tokenize import word_tokenize
    from sentence_transformers import SentenceTransformer
    from pypdf import PdfReader
    from qdrant_client import QdrantClient
    from qdrant_client.http.models import Distance, VectorParams
    from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
    from sklearn.metrics.pairwise import cosine_similarity
    from unstructured.partition.auto import partition
    from io import BytesIO
    from tqdm import tqdm
    print("Tüm kütüphaneler başarıyla import edildi!")
except Exception as e:
    print(f"Import sırasında bir hata oluştu: {e}")
    import traceback
    traceback.print_exc()

TensorFlow'u devre dışı bırakma ayarları yapıldı.
USE_TF: 0
TRANSFORMERS_NO_TF: 1
Tüm kütüphaneler başarıyla import edildi!


In [4]:
# --- 1. Doküman Parsers ---

# Logging ayarları
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- 1. Doküman Parsers ---

def parse_with_unstructured(path):
    """
    Belirtilen yoldaki dokümanı unstructured kütüphanesi ile ayrıştırır.
    PDF, DOCX, TXT ve daha birçok formatı destekler.
    """
    try:
        elements = partition(filename=path)
        # Elementler genellikle Text, NarrativeText, Title gibi türlerdedir.
        # Basitçe tüm metin içeriklerini birleştirelim.
        return "\n".join(str(el.text) for el in elements if hasattr(el, 'text') and el.text)
    except Exception as e:
        logging.error(f"Unstructured ile '{path}' ayrıştırılırken hata: {e}")
        # İsteğe bağlı: Hata durumunda belirli ayrıştırıcılara geri dönebilirsiniz.
        # Örneğin, eğer PDF ise ve unstructured başarısız olursa parse_pdf_pypdf'i deneyebilirsiniz.
        if path.lower().endswith(".pdf"):
            logging.info(f"Unstructured başarısız oldu, '{path}' için pypdf deneniyor...")
            return parse_pdf_pypdf(path) # Yedek pypdf ayrıştırıcısı
        elif path.lower().endswith(".docx"):
            logging.info(f"Unstructured başarısız oldu, '{path}' için python-docx deneniyor...")
            return parse_docx_python_docx(path) # Yedek python-docx ayrıştırıcısı
        elif path.lower().endswith(".txt"):
             logging.info(f"Unstructured başarısız oldu, '{path}' için yerel TXT okuyucu deneniyor...")
             return parse_txt_native(path) # Yedek TXT ayrıştırıcısı
        return None

def parse_pdf_pypdf(path):
    """pypdf kullanarak PDF dosyasını ayrıştırır (unstructured'a yedek)."""
    try:
        reader = PdfReader(path)
        text_parts = []
        for page in reader.pages:
            extracted_text = page.extract_text()
            if extracted_text: # Boş metinleri ekleme
                text_parts.append(extracted_text)
        return "\n".join(text_parts) if text_parts else None
    except Exception as e:
        logging.error(f"pypdf ile PDF '{path}' ayrıştırılırken hata: {e}")
        return None

def parse_docx_python_docx(path):
    """python-docx kullanarak DOCX dosyasını ayrıştırır (unstructured'a yedek)."""
    try:
        doc = docx.Document(path)
        return "\n".join(p.text for p in doc.paragraphs if p.text)
    except Exception as e:
        logging.error(f"python-docx ile DOCX '{path}' ayrıştırılırken hata: {e}")
        return None

def parse_txt_native(path):
    """Standart Python ile TXT dosyasını ayrıştırır (unstructured'a yedek)."""
    try:
        with open(path, encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        logging.error(f"TXT '{path}' okunurken hata: {e}")
        return None

In [5]:
def process_file(path, use_unstructured_primarily):
    content = None
    if use_unstructured_primarily:
        content = parse_with_unstructured(path)
    else:
        if path.lower().endswith(".pdf"):
            content = parse_pdf_pypdf(path)
        elif path.lower().endswith(".docx"):
            content = parse_docx_python_docx(path)
        elif path.lower().endswith(".txt"):
            content = parse_txt_native(path)
    return content


In [6]:
def load_documents(source_path, use_unstructured_primarily=False):
    """
    Bir klasör ya da .zip dosyasından belgeleri okur.
    PDF, DOCX ve TXT belgeleri desteklenmektedir.
    """
    docs = {}
    supported_extensions = (".pdf", ".docx", ".txt")

    # Kaynak klasör mü yoksa zip mi kontrolü
    if os.path.isdir(source_path):
        # Klasörse, eski yöntemi uygula
        search_pattern = os.path.join(source_path, "**", "*")
        for path in glob.glob(search_pattern, recursive=True):
            if os.path.isfile(path) and path.lower().endswith(supported_extensions):
                logging.info(f"İşleniyor: {path}")
                content = process_file(path, use_unstructured_primarily)
                if content:
                    docs[path] = content
                    logging.info(f"'{os.path.basename(path)}' başarıyla yüklendi. Uzunluk: {len(content)} karakter.")
                else:
                    logging.warning(f"'{os.path.basename(path)}' için içerik ayrıştırılamadı veya boş.")
    elif zipfile.is_zipfile(source_path):
        with zipfile.ZipFile(source_path, 'r') as zip_ref:
            for file_name in zip_ref.namelist():
                if file_name.lower().endswith(supported_extensions) and not file_name.endswith('/'):
                    try:
                        with zip_ref.open(file_name) as file:
                            raw_data = file.read()
                            temp_path = f"./tmp_{os.path.basename(file_name)}"
                            with open(temp_path, "wb") as temp_file:
                                temp_file.write(raw_data)
                            content = process_file(temp_path, use_unstructured_primarily)
                            if content:
                                docs[file_name] = content
                                logging.info(f"'{file_name}' başarıyla yüklendi. Uzunluk: {len(content)} karakter.")
                            else:
                                logging.warning(f"'{file_name}' için içerik ayrıştırılamadı.")
                    except Exception as e:
                        logging.error(f"{file_name} okunurken hata oluştu: {e}")
    else:
        raise ValueError("Verilen kaynak klasör veya .zip dosyası olmalıdır.")

    return docs

In [8]:
 # --- Örnek Kullanım Parsers ---
"""from unstructured.partition.auto import partition  # colab dışında çalıştırılırken yorum satırından kaldırılacak
documents_folder_path = "dosya yoluuuu \\AutoRAG-End-to-End-RAG-Framework-with-Turkish-Embedding-Evaluation\\wikipedia_documents.zip"
all_documents = load_documents(documents_folder_path, use_unstructured_primarily=True)

if not all_documents:
    print(f"'{documents_folder_path}' klasöründe (ve alt klasörlerinde) desteklenen formatta doküman bulunamadı veya işlenemedi.")
else:
    print(f"\nToplam {len(all_documents)} doküman bulundu ve işlendi:")
    for path, text_content in all_documents.items():
        print(f"\n--- Doküman: {path} ---")
        # İlk 200 karakteri göster
        print((text_content[:200] + "...") if text_content and len(text_content) > 200 else text_content)"""


'from unstructured.partition.auto import partition  # colab dışında çalıştırılırken yorum satırından kaldırılacak\ndocuments_folder_path = "dosya yoluuuu \\AutoRAG-End-to-End-RAG-Framework-with-Turkish-Embedding-Evaluation\\wikipedia_documents.zip"\nall_documents = load_documents(documents_folder_path, use_unstructured_primarily=True)\n\nif not all_documents:\n   print(f"\'{documents_folder_path}\' klasöründe (ve alt klasörlerinde) desteklenen formatta doküman bulunamadı veya işlenemedi.")\nelse:\n   print(f"\nToplam {len(all_documents)} doküman bulundu ve işlendi:")\n   for path, text_content in all_documents.items():\n       print(f"\n--- Doküman: {path} ---")\n       # İlk 200 karakteri göster\n       print((text_content[:200] + "...") if text_content and len(text_content) > 200 else text_content)'

In [7]:
from sentence_transformers import SentenceTransformer

matryoshka_dim = 768

embed_model = SentenceTransformer("Trendyol/TY-ecomm-embed-multilingual-base-v1.2.0", trust_remote_code=True, truncate_dim=matryoshka_dim)


2025-07-29 23:49:53,385 - INFO - Use pytorch device_name: cpu
2025-07-29 23:49:53,386 - INFO - Load pretrained SentenceTransformer: Trendyol/TY-ecomm-embed-multilingual-base-v1.2.0
2025-07-29 23:50:00,721 - WARNING - WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.7.1+cpu)
    Python  3.10.11 (you have 3.10.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
c:\Users\memo_\AppData\Local\Programs\Python\Python310\lib\site-packages\xformers\ops\swiglu_op.py:107: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(cls, ctx, x, w1, b1, w2, b2, w3, b3):
c:\Users\memo_\AppData\Local\Programs\Python\Python310\lib\site-packages\xformers\ops

In [8]:
import stanza

stanza.download('tr')  # 'tr' = Türkçe

nlp_tr = stanza.Pipeline('tr')

2025-07-29 23:50:03,773 - INFO - Downloaded file to C:\Users\memo_\stanza_resources\resources.json
2025-07-29 23:50:03,816 - INFO - Downloading default packages for language: tr (Turkish) ...
2025-07-29 23:50:04,785 - INFO - File exists: C:\Users\memo_\stanza_resources\tr\default.zip
2025-07-29 23:50:07,251 - INFO - Finished downloading models and saved to C:\Users\memo_\stanza_resources
2025-07-29 23:50:07,252 - INFO - Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-07-29 23:50:07,458 - INFO - Downloaded file to C:\Users\memo_\stanza_resources\resources.json
2025-07-29 23:50:08,310 - INFO - Loading these models for language: tr (Turkish):
| Processor | Package       |
-----------------------------
| tokenize  | imst          |
| mwt       | imst          |
| pos       | imst_charlm   |
| lemma     | imst_nocharlm |
| depparse  | imst_charlm  

In [9]:
def create_semantic_chunks_stanza(
    text_content: str,
    embed_model_instance,  # SentenceTransformer modeli
    stanza_nlp_model,  # Yüklenmiş Stanza modeli (nlp_tr)
    pre_segment_max_chars: int = 600,  # Ön segmentlerin maksimum karakter uzunluğu
    pre_segment_max_sentences: int = 6,  # Ön segmentlerdeki maksimum cümle sayısı
    similarity_threshold: float = 0.40,  # Nihai chunk'lar için benzerlik eşiği (0-1 arası)
    final_chunk_min_pre_segments: int = 1,  # Bir nihai chunk'ta en az kaç ön-segment olmalı
    final_chunk_max_pre_segments: int = 6,  # Bir nihai chunk'ta en fazla kaç ön-segment olmalı
):
    if not text_content or not text_content.strip() or not stanza_nlp_model:
        return []

    # 1. Adım: Metni Stanza ile cümlelere böl
    doc = stanza_nlp_model(text_content)
    sentences = [" ".join([word.text for word in sentence.words]) for sentence in doc.sentences]

    if not sentences:
        return []

    # 2. Adım: Cümleleri "ön-segmentlere" birleştir
    pre_segments = []
    current_pre_segment_sentences = []
    current_pre_segment_chars = 0
    for sentence in sentences:
        # Ön segment için karakter veya cümle limitine ulaşıldı mı?
        if (current_pre_segment_chars + len(sentence) > pre_segment_max_chars and current_pre_segment_sentences) or \
           (len(current_pre_segment_sentences) >= pre_segment_max_sentences):
            pre_segments.append(" ".join(current_pre_segment_sentences))
            current_pre_segment_sentences = []
            current_pre_segment_chars = 0

        current_pre_segment_sentences.append(sentence)
        current_pre_segment_chars += len(sentence)

    # Kalan son ön-segmenti ekle
    if current_pre_segment_sentences:
        pre_segments.append(" ".join(current_pre_segment_sentences))

    if not pre_segments:
        return []

    # 3. Adım: Ön-segmentlerin embedding'lerini al
    if len(pre_segments) == 1:  # Sadece bir ön-segment varsa, onu direkt chunk olarak döndür
        return pre_segments

    pre_segment_embeddings = embed_model.encode(pre_segments, convert_to_numpy=True)

    # 4. Adım: Ön-segmentleri anlamsal olarak nihai chunk'lara birleştir
    final_chunks = []
    # Metinleri ve embedding indekslerini saklamak için listeler
    current_final_chunk_pre_segments_texts = [pre_segments[0]]
    # current_final_chunk_pre_segments_indices = [0] # İndeksler şu an direkt kullanılmıyor ama ileride lazım olabilir

    for i in range(1, len(pre_segments)):
        emb_current_pre_segment = pre_segment_embeddings[i]
        emb_previous_pre_segment = pre_segment_embeddings[i-1] # Bir önceki ön-segmentle karşılaştır

        # Kosinüs benzerliğini hesapla
        similarity_score = cosine_similarity([emb_current_pre_segment], [emb_previous_pre_segment])[0][0]
        # similarity_score = np.clip(similarity_score, -1.0, 1.0) # İsteğe bağlı ekstra güvenlik

        split_condition_met = False

        # Benzerlik, belirlenen eşiğin ALTINDAYSA yeni bir chunk başlat
        if similarity_score < similarity_threshold:
            split_condition_met = True

        # Nihai chunk için maksimum ön-segment sayısına ulaşıldı mı?
        if len(current_final_chunk_pre_segments_texts) >= final_chunk_max_pre_segments:
            split_condition_met = True

        # Bölme koşulu sağlandıysa ve mevcut chunk minimum boyuttaysa, chunk'ı sonlandır
        if split_condition_met and len(current_final_chunk_pre_segments_texts) >= final_chunk_min_pre_segments:
            final_chunks.append(" ".join(current_final_chunk_pre_segments_texts))
            current_final_chunk_pre_segments_texts = [pre_segments[i]] # Yeni chunk bu ön-segmentle başlar
            # current_final_chunk_pre_segments_indices = [i]
        else:
            # Bölme koşulu sağlanmadıysa veya chunk minimum boyutta değilse, mevcut chunk'a ekle
            current_final_chunk_pre_segments_texts.append(pre_segments[i])
            # current_final_chunk_pre_segments_indices.append(i)

    # Döngü bittikten sonra kalan son nihai chunk'ı ekle
    if current_final_chunk_pre_segments_texts and len(current_final_chunk_pre_segments_texts) >= final_chunk_min_pre_segments:
        final_chunks.append(" ".join(current_final_chunk_pre_segments_texts))

    return [chunk for chunk in final_chunks if chunk.strip()] # Boş chunk'ları temizle

In [10]:
DATA_FOLDER = "C:/Users/memo_/Downloads/extracted_wikidump_data/AA"

In [12]:
import json

all_documents = {}
# --- Ana döngü: her dosya için çalış ---
for filename in sorted(os.listdir(DATA_FOLDER))[:10]:
    if not filename.startswith("wiki_"):
        continue

    file_path = os.path.join(DATA_FOLDER, filename)
    collection_name = os.path.splitext(filename)[0]  # wiki_00

    print(f"\n[+] İşleniyor: {filename} → koleksiyon: {collection_name}")

    with open(file_path, "r", encoding="utf-8") as file:
        for line_idx, line in enumerate(file):
            try:
                data = json.loads(line)
                key = f"{filename}_line_{line_idx}"
                all_documents[key] = {
                    "title": data.get("title", ""),
                    "url": data.get("url", ""),
                    "text": data.get("text", "")
                }
            except Exception as e:
                print(f"[!] Satır {line_idx} okunamadı: {e}")


[+] İşleniyor: wiki_00 → koleksiyon: wiki_00

[+] İşleniyor: wiki_01 → koleksiyon: wiki_01

[+] İşleniyor: wiki_02 → koleksiyon: wiki_02

[+] İşleniyor: wiki_03 → koleksiyon: wiki_03

[+] İşleniyor: wiki_04 → koleksiyon: wiki_04

[+] İşleniyor: wiki_05 → koleksiyon: wiki_05

[+] İşleniyor: wiki_06 → koleksiyon: wiki_06

[+] İşleniyor: wiki_07 → koleksiyon: wiki_07

[+] İşleniyor: wiki_08 → koleksiyon: wiki_08

[+] İşleniyor: wiki_09 → koleksiyon: wiki_09


In [14]:
print(len(all_documents))

1838


In [15]:
from collections import defaultdict

file_counts = defaultdict(int)
for key in all_documents:
    file_name = key.split("_line_")[0]
    file_counts[file_name] += 1

for file_name, count in sorted(file_counts.items()):
    print(f"{file_name}: {count} belge")


wiki_00: 44 belge
wiki_01: 67 belge
wiki_02: 91 belge
wiki_03: 108 belge
wiki_04: 71 belge
wiki_05: 209 belge
wiki_06: 1018 belge
wiki_07: 75 belge
wiki_08: 100 belge
wiki_09: 55 belge


In [ ]:
"""# --- 2. Semantic Chunking ---
chunks_text_list = []
metadata_for_qdrant = []
global_chunk_id_counter = 1

for path, text_content in all_documents.items():
    text = text_content.get("text", "")
    if not text or not text.strip():
        print(f"Uyarı: '{path}' dosyası boş veya sadece boşluk içeriyor.")
        continue

    if nlp_tr is None: 
        print(f"Stanza modeli yüklenemediği için '{path}' üzerinde semantic chunking atlanıyor.")
        continue

    current_doc_chunks = create_semantic_chunks_stanza(
        text_content=text,
        embed_model_instance=embed_model,
        stanza_nlp_model=nlp_tr,
        pre_segment_max_chars=600,
        pre_segment_max_sentences=6,
        similarity_threshold=0.40,
        final_chunk_min_pre_segments=1,
        final_chunk_max_pre_segments=6
    )

    if not current_doc_chunks:
        print(f"Uyarı: '{path}' dosyasından semantic chunking ile hiç chunk üretilemedi.")
        continue

    print(f"'{path}' dosyasından {len(current_doc_chunks)} adet semantic chunk üretildi.")

    chunk_id_in_file_counter = 0
    for chunk_text in current_doc_chunks:
        if not chunk_text or not chunk_text.strip():
            continue

        chunks_text_list.append(chunk_text)
        metadata_for_qdrant.append({
            "source": path,
            "chunk_id_in_file": chunk_id_in_file_counter,
            "original_text": chunk_text,
            "title": text_content.get("title", ""),
            "url": text_content.get("url", "")
        })
        global_chunk_id_counter += 1
        chunk_id_in_file_counter += 1


if not chunks_text_list:
    print("Hiç chunk üretilemedi. İşlem durduruluyor.")
    exit()

print(f"Toplam {len(chunks_text_list)} adet chunk metni oluşturuldu.")"""

Batches: 100%|██████████| 7/7 [00:12<00:00,  1.85s/it]


'wiki_00_line_0' dosyasından 66 adet semantic chunk üretildi.
'wiki_00_line_1' dosyasından 1 adet semantic chunk üretildi.


Batches: 100%|██████████| 2/2 [00:02<00:00,  1.50s/it]


'wiki_00_line_2' dosyasından 20 adet semantic chunk üretildi.


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]


...

'wiki_09_line_52' dosyasından 13 adet semantic chunk üretildi.


Batches: 100%|██████████| 2/2 [00:02<00:00,  1.34s/it]


'wiki_09_line_53' dosyasından 14 adet semantic chunk üretildi.


Batches: 100%|██████████| 4/4 [00:08<00:00,  2.20s/it]

'wiki_09_line_54' dosyasından 40 adet semantic chunk üretildi.
Toplam 5573 adet chunk metni oluşturuldu.


In [17]:
# Tüm chunk’lar için embedding üret
embeddings = embed_model.encode(chunks_text_list, show_progress_bar=True, convert_to_numpy=True)

Batches: 100%|██████████| 175/175 [07:02<00:00,  2.41s/it]


In [18]:
#!pip uninstall -y tensorflow tensorflow-cpu tensorflow-gpu tensorboard tb-nightly tensorflow-estimator tensorflow-io tensorflow-io-gcs-filesystem tf-keras keras

In [20]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# Qdrant Cloud endpoint ve API key

# --- Qdrant Bağlantısı ---
def connect_qdrant_local():
    """
    Lokal Docker Qdrant container'ına bağlanır.
    """
    client = QdrantClient(
        url="http://localhost:6333",  # Cloud URL yerine lokal
        prefer_grpc=False,
        timeout=60.0
    )
    return client

# Bağlantıyı kur
qdrant_client = connect_qdrant_local()

# Test: Mevcut koleksiyonları listele
print(qdrant_client.get_collections())

2025-07-20 18:35:28,776 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
C:\Users\memo_\AppData\Local\Temp\ipykernel_11916\4231166172.py:11: UserWarning: Qdrant client version 1.14.3 is incompatible with server version 1.12.6. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient(
2025-07-20 18:35:28,839 - INFO - HTTP Request: GET http://localhost:6333/collections "HTTP/1.1 200 OK"


collections=[CollectionDescription(name='semantic-rake-LLM-data_collection'), CollectionDescription(name='hybrid-2-pdf-4_data_collection'), CollectionDescription(name='jotform-pdf-syllabus_data_collection'), CollectionDescription(name='semantic-jotform-LLM_data_collection'), CollectionDescription(name='hybrid-2-pdf-3_data_collection'), CollectionDescription(name='hybrid-2-pdf-semant-scipy_data_collection'), CollectionDescription(name='hybrid-2-pdf-LLM-scipy_data_collection'), CollectionDescription(name='TBF-temsilci'), CollectionDescription(name='TBF-temsilci-2'), CollectionDescription(name='semantic-tfidf-LLM-data_collection'), CollectionDescription(name='Dynamic-jotform-pdf_data_collection'), CollectionDescription(name='hybrid-2-pdf-english-scipy_data_collection'), CollectionDescription(name='wiki_99'), CollectionDescription(name='jotform-pdf-english_data_collection'), CollectionDescription(name='jotform-pdf-LLM_data_collection'), CollectionDescription(name='jotform-ML-pdf_data_colle

In [21]:
# --- Qdrant Koleksiyon Oluşturma ve Upsert ---
vector_size = embeddings.shape[1]

try:
    qdrant_client.delete_collection(collection_name=collection_name)
    print(f"'{collection_name}' koleksiyonu silindi.")
except Exception:
    print(f"'{collection_name}' koleksiyonu silinemedi veya zaten yoktu.")

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)
print(f"'{collection_name}' koleksiyonu oluşturuldu/yeniden oluşturuldu.")

# Upsert işlemi
points = [
    {
        "id": idx,
        "vector": embeddings[idx].tolist(),
        "payload": metadata_for_qdrant[idx]
    }
    for idx in range(len(embeddings))
]

BATCH_SIZE = 100
uploaded_total = 0

for i in tqdm(range(0, len(points), BATCH_SIZE), desc="Uploading to Qdrant"):
    batch = points[i:i + BATCH_SIZE]
    try:
        qdrant_client.upsert(collection_name=collection_name, points=batch)
        uploaded_total += len(batch)
    except Exception as e:
        print(f"[!] Batch {i}-{i + BATCH_SIZE} yüklenemedi: {e}")
        time.sleep(2)

print(f"{uploaded_total} nokta başarıyla Qdrant'taki '{collection_name}' koleksiyonuna yüklendi.")

2025-07-20 18:35:32,744 - INFO - HTTP Request: DELETE http://localhost:6333/collections/wiki_09 "HTTP/1.1 200 OK"


'wiki_09' koleksiyonu silindi.


2025-07-20 18:35:33,273 - INFO - HTTP Request: PUT http://localhost:6333/collections/wiki_09 "HTTP/1.1 200 OK"


'wiki_09' koleksiyonu oluşturuldu/yeniden oluşturuldu.


Uploading to Qdrant:  11%|█         | 6/56 [00:00<00:06,  7.21it/s]2025-07-20 18:35:35,089 - INFO - HTTP Request: PUT http://localhost:6333/collections/wiki_09/points?wait=true "HTTP/1.1 200 OK"
2025-07-20 18:35:35,223 - INFO - HTTP Request: PUT http://localhost:6333/collections/wiki_09/points?wait=true "HTTP/1.1 200 OK"
Uploading to Qdrant:  43%|████▎     | 24/56 [00:03<00:04,  7.10it/s]2025-07-20 18:35:37,437 - INFO - HTTP Request: PUT http://localhost:6333/collections/wiki_09/points?wait=true "HTTP/1.1 200 OK"
2025-07-20 18:35:37,553 - INFO - HTTP Request: PUT http://localhost:6333/collections/wiki_09/points?wait=true "HTTP/1.1 200 OK"
Uploading to Qdrant:  80%|████████  | 45/56 [00:06<00:01,  8.61it/s]2025-07-20 18:35:40,265 - INFO - HTTP Request: PUT http://localhost:6333/collections/wiki_09/points?wait=true "HTTP/1.1 200 OK"
2025-07-20 18:35:40,377 - INFO - HTTP Request: PUT http://localhost:6333/collections/wiki_09/points?wait=true "HTTP/1.1 200 OK"
Uploading to Qdrant:  84%|███

5573 nokta başarıyla Qdrant'taki 'wiki_09' koleksiyonuna yüklendi.


In [12]:
import json
import os
import time
from qdrant_client import QdrantClient, models
from tqdm import tqdm


# --- 1. Qdrant Bağlantısı (Döngüden önce bir kez yapılır) ---
def connect_qdrant_local():
    """Lokal Docker Qdrant container'ına bağlanır."""
    try:
        client = QdrantClient(
            url="http://localhost:6333",
            prefer_grpc=False,
            timeout=60.0
        )
        client.get_collections()
        print("Qdrant'a başarıyla bağlanıldı.")
        return client
    except Exception as e:
        print(f"Qdrant'a bağlanılamadı: {e}")
        return None

qdrant_client = connect_qdrant_local()

if qdrant_client is None:
    exit()


try:
    existing_collections_response = qdrant_client.get_collections()
    existing_collections = {col.name for col in existing_collections_response.collections}
    print(f"Mevcut koleksiyonlar bulundu: {existing_collections}")
except Exception as e:
    print(f"Mevcut koleksiyonlar alınamadı: {e}")
    existing_collections = set()


for filename in sorted(os.listdir(DATA_FOLDER))[20:30]:
    if not filename.startswith("wiki_"):
        continue

    collection_name = os.path.splitext(filename+"trendyol")[0]
    

    if collection_name in existing_collections:
        print(f"\n[✓] Koleksiyon '{collection_name}' zaten mevcut. Atlanıyor: {filename}")
        continue


    file_path = os.path.join(DATA_FOLDER, filename)
    print(f"\n{'='*50}\n[+] İşleniyor: {filename} → Koleksiyon: {collection_name}\n{'='*50}")

    chunks_text_list = []
    metadata_for_qdrant = []
    
    with open(file_path, "r", encoding="utf-8") as file:
        for line_idx, line in enumerate(file):
            try:
                data = json.loads(line)
                text = data.get("text", "")
                
                if not text or not text.strip():
                    continue

                if nlp_tr is None: 
                    print(f"Stanza modeli yüklenemediği için semantic chunking atlanıyor.")
                    break

                current_doc_chunks = create_semantic_chunks_stanza(
                    text_content=text,
                    embed_model_instance=embed_model,
                    stanza_nlp_model=nlp_tr,
                    pre_segment_max_chars=600,
                    pre_segment_max_sentences=6,
                    similarity_threshold=0.40,
                    final_chunk_min_pre_segments=1,
                    final_chunk_max_pre_segments=6
                )

                if not current_doc_chunks:
                    continue

                for chunk_text in current_doc_chunks:
                    if not chunk_text or not chunk_text.strip():
                        continue
                    chunks_text_list.append(chunk_text)
                    metadata_for_qdrant.append({
                        "source": f"{filename}_line_{line_idx}",
                        "original_text": chunk_text,
                        "title": data.get("title", ""),
                        "url": data.get("url", "")
                    })

            except Exception as e:
                print(f"[!] {filename} - Satır {line_idx} işlenemedi: {e}")

    if not chunks_text_list:
        print(f"'{filename}' dosyasından hiç chunk üretilemedi. Sonraki dosyaya geçiliyor.")
        continue

    print(f"'{filename}' için toplam {len(chunks_text_list)} adet chunk oluşturuldu. Embedding'ler üretiliyor...")

    embeddings = embed_model.encode(chunks_text_list, show_progress_bar=True, convert_to_numpy=True)
    vector_size = embeddings.shape[1]

    # Koleksiyonu silme komutu artık gerekli değil, çünkü zaten var olmadığını biliyoruz.
    # try:
    #     qdrant_client.delete_collection(collection_name=collection_name)
    # except Exception:
    #     pass

    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=vector_size, distance=models.Distance.COSINE),
    )
    print(f"'{collection_name}' koleksiyonu oluşturuldu.")

    points = [
        models.PointStruct(
            id=idx,
            vector=embeddings[idx].tolist(),
            payload=metadata_for_qdrant[idx]
        )
        for idx in range(len(embeddings))
    ]

    print(f"'{collection_name}' koleksiyonuna {len(points)} nokta yükleniyor...")
    qdrant_client.upsert(
        collection_name=collection_name,
        points=points,
        wait=True
    )

    print(f"'{filename}' dosyasından gelen {len(points)} nokta başarıyla '{collection_name}' koleksiyonuna yüklendi.")

print("\nTüm dosyaların işlenmesi tamamlandı.")

2025-07-30 10:59:12,869 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
C:\Users\memo_\AppData\Local\Temp\ipykernel_15268\200918423.py:12: UserWarning: Qdrant client version 1.14.3 is incompatible with server version 1.12.6. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient(
2025-07-30 10:59:12,956 - INFO - HTTP Request: GET http://localhost:6333/collections "HTTP/1.1 200 OK"
2025-07-30 10:59:13,024 - INFO - HTTP Request: GET http://localhost:6333/collections "HTTP/1.1 200 OK"


Qdrant'a başarıyla bağlanıldı.
Mevcut koleksiyonlar bulundu: {'wiki_29trendyol', 'wiki_04', 'wiki_24trendyol', 'wiki_15', 'wiki_32', 'wiki_14', 'wiki_11', 'wiki_38', 'wiki_16trendyol', 'wiki_07trendyol', 'wiki_27trendyol', 'wiki_01', 'wiki_03', 'wiki_16', 'wiki_34', 'wiki_24', 'wiki_22trendyol', 'wiki_20', 'wiki_03trendyol', 'wiki_11trendyol', 'wiki_05trendyol', 'wiki_04trendyol', 'wiki_00', 'wiki_10trendyol', 'wiki_06trendyol', 'bahaddin_eravci', 'wiki_18', 'wiki_08trendyol', 'wiki_14trendyol', 'wiki_22', 'wiki_06', 'wiki_36', 'wiki_31', 'wiki_25trendyol', 'wiki_30', 'wiki_10', 'bahaddin_eravci 1', 'wiki_12trendyol', 'wiki_18trendyol', 'wiki_13', 'wiki_37', 'wiki_09trendyol', 'wiki_39', 'wiki_05', 'wiki_21', 'wiki_44', 'llmfao_collection', 'wiki_42', 'wiki_20trendyol', 'wiki_19trendyol', 'wiki_41', 'wiki_13trendyol', 'wiki_23trendyol', 'wiki_21trendyol', 'wiki_02', 'wiki_17', 'wiki_15trendyol', 'wiki_combined', 'wiki_07', 'wiki_35', 'wiki_17trendyol', 'wiki_29', 'wiki_01trendyol', 'wi

In [2]:
from qdrant_client import QdrantClient, models
from tqdm import tqdm
from uuid import uuid4

# -------------------- Ayarlar --------------------
SOURCE_COLLECTIONS = [f"wiki_{str(i).zfill(2)}trendyol" for i in range(30)]
TARGET_COLLECTION = "wiki_combined_trendyol"

client = QdrantClient("http://localhost:6333")

# --- Eğer hedef koleksiyon yoksa oluştur ---
existing = client.get_collections().collections
existing_names = [col.name for col in existing]

if TARGET_COLLECTION not in existing_names:
    sample_info = client.get_collection(collection_name=SOURCE_COLLECTIONS[0])
    vector_size = sample_info.config.params.vectors.size

    client.create_collection(
        collection_name=TARGET_COLLECTION,
        vectors_config=models.VectorParams(size=vector_size, distance=models.Distance.COSINE),
    )
    print(f"[+] '{TARGET_COLLECTION}' adlı birleşik koleksiyon oluşturuldu.")
else:
    print(f"[✓] '{TARGET_COLLECTION}' koleksiyonu zaten mevcut. Veri eklenmeye devam edilecek.")

# -------------------- Kopyalama Döngüsü --------------------
for col in tqdm(SOURCE_COLLECTIONS, desc="Koleksiyonlar Aktarılıyor"):
    offset = 0
    limit = 100
    
    while True:
        response = client.scroll(
            collection_name=col,
            offset=offset,
            limit=limit,
            with_vectors=True,
            with_payload=True
        )

        if not response[0]:
            break  # Daha fazla veri yok

        points = [
            models.PointStruct(
                id=str(uuid4()),  # benzersiz ID
                vector=point.vector,
                payload={
                    **point.payload,  # tüm orijinal metadata'lar (title, url, text vs.)
                    "source_collection": col  # İsteğe bağlı: hangi koleksiyondan geldi
                }
            )
            for point in response[0]
        ]

        client.upsert(
            collection_name=TARGET_COLLECTION,
            points=points,
            wait=True
        )

        offset += len(points)

print("\n✅ Tüm koleksiyonlar, metadata'larıyla birlikte başarıyla 'wiki_combined' içine kopyalandı.")


C:\Users\memo_\AppData\Local\Temp\ipykernel_5728\442279460.py:9: UserWarning: Qdrant client version 1.14.3 is incompatible with server version 1.12.6. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient("http://localhost:6333")


[✓] 'wiki_combined_trendyol' koleksiyonu zaten mevcut. Veri eklenmeye devam edilecek.


Koleksiyonlar Aktarılıyor: 100%|██████████| 30/30 [01:17<00:00,  2.60s/it]


✅ Tüm koleksiyonlar, metadata'larıyla birlikte başarıyla 'wiki_combined' içine kopyalandı.


In [1]:
try:
    collection_info = qdrant_client.get_collection(collection_name=collection_name)
    print(f"Koleksiyon Bilgileri '{collection_name}':")
    print(f"  Durum: {collection_info.status}")
    print(f"  Nokta Sayısı: {collection_info.points_count}")
    print(f"  Vektör Boyutu: {collection_info.config.params.vectors.size}")
    print(f"  Uzaklık Metriği: {collection_info.config.params.vectors.distance}")
except Exception as e:
    print(f"Koleksiyon bilgileri alınırken hata oluştu: {e}")

Koleksiyon bilgileri alınırken hata oluştu: name 'qdrant_client' is not defined


In [ ]:
all_points = []
next_page_offset = None  # Başlangıçta offset yok

print(f"\n'{collection_name}' koleksiyonundaki noktalar (chunk'lar) çekiliyor...")

try:
    while True:
        # Scroll ile bir sonraki sayfayı al
        # limit: Her seferinde kaç nokta çekileceği
        # with_payload=True: Payload'ı da al (metin ve metadata burada)
        # with_vectors=False: Vektörleri çekmek istemiyorsanız (daha hızlı olabilir)
        records, next_page_offset = qdrant_client.scroll(
            collection_name=collection_name,
            limit=100,  # Her seferinde 100 nokta çek
            offset=next_page_offset,
            with_payload=True,
            with_vectors=False # Vektörlere ihtiyacınız yoksa False yapın
        )

        if not records:
            # Daha fazla nokta kalmadıysa döngüden çık
            break

        all_points.extend(records)

        # Eğer bir sonraki sayfa için offset None ise (son sayfa demek olabilir)
        if next_page_offset is None:
            break

    print(f"Toplam {len(all_points)} nokta (chunk) çekildi.")

    # İlk birkaç chunk'ı (payload'ını) yazdıralım
    for i, point in enumerate(all_points[:10]): # İlk 5 tanesini göster
        print(f"\n--- Chunk {i+1} (ID: {point.id}) ---")
        if point.payload:
            print(f"  Kaynak: {point.payload.get('source', 'N/A')}")
            print(f"  Dosya İçi Chunk ID: {point.payload.get('chunk_id_in_file', 'N/A')}")
            print(f"  Orijinal Metin: {point.payload.get('original_text', 'N/A')[:200]}...") # Metnin ilk 200 karakteri
        else:
            print("  Payload bulunamadı.")

except Exception as e:
    print(f"Noktalar çekilirken hata oluştu: {e}")
    import traceback
    traceback.print_exc()

2025-07-08 15:30:25,421 - INFO - HTTP Request: POST https://d8f9c106-88a7-4933-bb28-d6b7b23bdc02.us-east4-0.gcp.cloud.qdrant.io:6333/collections/llmfao_collection/points/scroll "HTTP/1.1 200 OK"



'llmfao_collection' koleksiyonundaki noktalar (chunk'lar) çekiliyor...


2025-07-08 15:30:25,873 - INFO - HTTP Request: POST https://d8f9c106-88a7-4933-bb28-d6b7b23bdc02.us-east4-0.gcp.cloud.qdrant.io:6333/collections/llmfao_collection/points/scroll "HTTP/1.1 200 OK"
2025-07-08 15:30:26,038 - INFO - HTTP Request: POST https://d8f9c106-88a7-4933-bb28-d6b7b23bdc02.us-east4-0.gcp.cloud.qdrant.io:6333/collections/llmfao_collection/points/scroll "HTTP/1.1 200 OK"
2025-07-08 15:30:26,224 - INFO - HTTP Request: POST https://d8f9c106-88a7-4933-bb28-d6b7b23bdc02.us-east4-0.gcp.cloud.qdrant.io:6333/collections/llmfao_collection/points/scroll "HTTP/1.1 200 OK"
2025-07-08 15:30:26,427 - INFO - HTTP Request: POST https://d8f9c106-88a7-4933-bb28-d6b7b23bdc02.us-east4-0.gcp.cloud.qdrant.io:6333/collections/llmfao_collection/points/scroll "HTTP/1.1 200 OK"
2025-07-08 15:30:26,622 - INFO - HTTP Request: POST https://d8f9c106-88a7-4933-bb28-d6b7b23bdc02.us-east4-0.gcp.cloud.qdrant.io:6333/collections/llmfao_collection/points/scroll "HTTP/1.1 200 OK"
2025-07-08 15:30:26,809 -

Toplam 939 nokta (chunk) çekildi.

--- Chunk 1 (ID: 0) ---
  Kaynak: wikipedia_documents/wikipedia_500.docx
  Dosya İçi Chunk ID: 0
  Orijinal Metin: Henry Edwards ( entomolog ) " Harry" olarak bilinen Henry Edwards ( 27 Ağustos 1827 - 9 Haziran 1891 ) , tiyatro çalışmalarıyla Avustralya , San Francisco ve New York'ta ün kazanan bir İngiliz tiyatro...

--- Chunk 2 (ID: 1) ---
  Kaynak: wikipedia_documents/wikipedia_500.docx
  Dosya İçi Chunk ID: 1
  Orijinal Metin: Avustralya'ya yelken açtıktan sonra Edwards , özellikle Melbourne ve Sidney'de Shakespeare oyunlarında ve hafif komedilerde profesyonel olarak rol aldı . İngiltere'de ki çocukluğu ve Avustrala'da ki o...

--- Chunk 3 (ID: 2) ---
  Kaynak: wikipedia_documents/wikipedia_500.docx
  Dosya İçi Chunk ID: 2
  Orijinal Metin: Edwards , San Francisco'da Bohemian Club'ın kurucu üyelerinden biri ydi ve kendisinin onuruna düzenlenen bir etkinlik sonrası , kulübün Bohemian Grove'da ki geleneksel yaz kampını hayata geçirildi . B...

--- C

In [23]:
# --- 5. Retrieve Fonksiyonu ---
def retrieve_top_k(query: str, k: int = 5):
    q_emb = embed_model.encode([query], convert_to_numpy=True)[0]
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=q_emb.tolist(),
        limit=k,
        with_payload=True
    )
    results = []
    for hit in search_result:
        payload = hit.payload
        score = hit.score
        # idx = hit.id # Bu ID, Qdrant'a yüklerken verdiğiniz ID (0, 1, 2...)

        src = payload.get("source", "<unknown_source>")
        # chunk_id_in_file, payload'da sakladığınız isimle eşleşmeli
        chunk_id_file = payload.get("chunk_id_in_file", -1)

        # En önemlisi: Metni payload'dan alıyoruz!
        text = payload.get("original_text", "<text_not_found_in_payload>")

        results.append({
            "source": src,
            "chunk_id_in_file": chunk_id_file,
            "score": score,
            "text": text  # Payload'dan gelen metin
        })
    return results

In [ ]:
"""# --- 6. Hugging Face LLM Entegrasyonu ---

model_id = "ytu-ce-cosmos/Turkish-Gemma-9b-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

# 1) Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 2) Model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    max_memory={
        0: "20GB",    # GPU: 20 GB
        "cpu": "48GB" # CPU: geri kalan
    },
)

# 3) Pipeline – mutlaka burada tanımla
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)
"""

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [24]:
import requests

def ollama_generate(prompt, model="gemma:2b", max_tokens=1024, temperature=0.6):
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": temperature,
                    "num_predict": max_tokens,
                    "top_p": 0.9
                }
            }
        )

        if response.status_code == 200:
            json_data = response.json()
            return json_data.get("response", "[cevap bulunamadı]")
        else:
            print("Hata:", response.text)
            return "[API Hatası]"

    except Exception as e:
        print("İstisna oluştu:", e)
        return "[HATA]"


In [25]:
def rag_answer(query: str):
    # Retrieve (aynı kalıyor)
    results = retrieve_top_k(query, k=5)
    print("Toplam bulunan kaynak sayısı:", len(results)) 
    if not results:
        print("Uyarı: Qdrant veri tabanı bu sorguya sonuç döndürmedi.")
        return "[Bilgi bulunamadı]"
    # Kaynak metinleri oluştur
    sources_text = "\n\n".join(
        f"[{i+1}] Skor: {r['score']:.4f}\n{r['text']}"
        for i, r in enumerate(results)
    )

    # Prompt
    prompt = (
        "Sen bir Türkçe NLP uzmanısın. Aşağıdaki kaynaklara dayanarak soruyu yanıtla:\n\n"
        f"Soru: {query}\n\n"
        f"Kaynaklar:\n{sources_text}\n\n"
        "Cevabı açıkla:"
    )
    print("Oluşan prompt:\n", prompt)

    answer = ollama_generate(prompt)
    
    return answer


In [ ]:
"""#gemma 9b için

def rag_answer(query: str):
    # Retrieve
    results = retrieve_top_k(query, k=5)
    # Kaynak metinleri oluştur
    sources_text = "\n\n".join(
        f"[{i+1}] Skor: {r['score']:.4f}\n{r['text']}"
        for i, r in enumerate(results)
    )
    # Prompt
    prompt = (
        "Sen Türkçe NLP uzmanısın. Aşağıdaki kaynaklara dayanarak soruyu yanıtla:\n\n"
        f"Soru: {query}\n\n"
        f"Kaynaklar:\n{sources_text}\n\n"
        "Cevabı açıkla:"
    )

    # Cevap üret
    output = text_generator(
        prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    return output[0]["generated_text"]"""

In [26]:
# --- 7. Örnek Kullanım ---
if __name__ == "__main__":
    question = "Henry Edwards kimdir?"
    answer = rag_answer(question)
    print(answer)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]
C:\Users\memo_\AppData\Local\Temp\ipykernel_11916\3056576069.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(
2025-07-20 18:36:55,778 - INFO - HTTP Request: POST http://localhost:6333/collections/wiki_09/points/search "HTTP/1.1 200 OK"


Toplam bulunan kaynak sayısı: 5
Oluşan prompt:
 Sen bir Türkçe NLP uzmanısın. Aşağıdaki kaynaklara dayanarak soruyu yanıtla:

Soru: Henry Edwards kimdir?

Kaynaklar:
[1] Skor: 0.4825
John Reid Edwards ( 10 Haziran 1953 ) , Kuzey Karolina eski senatörü . 2004'te ki başkanlık seçimlerinde John Kerry'nin başkan yardımcısı adayı idi . İlk yılları . John Edwards Güney Karolina eyaletinin Seneca ilinde doğdu , sonra Kuzey Karolina eyaletinde Robbins'te yaşadı . Babası bir tekstil çalışanı ydı , annesi ise postanede çalışıyordu . Ailesinde üniversiteye giden ilk kişi ydi . Güney Karolina'da Clemson University'de Amerikan Futbolu bursu ile okudu , ardından da North Carolina State University'ye geçti . Tekstil teknolojisi branşında 1974'te mezun olan Edwards , University of North Carolina at Chapel Hill'de hukuk eğitimi aldı . Bu son iki üniversitede üstün başarı ile şeref listesine girdi . Hukuk okurken hukuk öğrencisi Elizabeth Anania ile tanıştı ve evlendi . Hukuk kariyeri . Politikaya atılm